In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
import nltk
import re

import boto3
from botocore.exceptions import ClientError
from io import StringIO

In [ ]:
class Streaming:
    def __init__(self):
        self.message = 'This is init'
        
    def createClient(self, service, region):
        return boto3.client(service, region_name = region)
    
    def loadData(self):
        bucket1 = 's3://hiringtrendanalysis/wordcount.csv' # already created on S3
        client = boto3.resource('s3')
        df_data = pd.read_csv(bucket1)
        return df_data
    
    def sendKinesis(self,kinesis_client, kinesis_stream_name, kinesis_shard_count, data):
        kinesis_records = []
        
        (rows, columns) = data.shape
        
        currentBytes = 0
        
        rowCount = 0
        
        totalRowCount = rows
        
        sendKinesis = False
        
        shardCount = 1
        
        for index, row in data.iterrows():
            values = '|'.join(str(value) for value in row)
#             print(values)
            
            encodedValues = bytes(values, 'utf-8')
            
            kinesis_record = {"Data": encodedValues, 
                             "PartitionKey" : str(shardCount)}
            
            kinesis_records.append(kinesis_record)
            
            stringBytes = len(values.encode('utf-8'))
            currentBytes = currentBytes + stringBytes 
            
            if len(kinesis_records) == 500:
                sendKinesis = True
                
            if currentBytes > 50000:
                sendKinesis = True
            
            
            if rowCount == totalRowCount:
                sendKinesis = True
                
            if sendKinesis == True:
                
                response = kinesis_client.put_records(Records = kinesis_records,   StreamName = kinesis_stream_name )
            
                kinesis_records = []
                sendKinesis = False
                currentBytes = 0
                
                shardCount = shardCount + 1
                
                if shardCount > kinesis_shard_count:
                    shardCount = 1
                    
            rowCount += 1
            
            # log out how many records were pushed
        print('Total Records sent to Kinesis: {0}'.format(totalRowCount))
       

In [ ]:
if __name__ == '__main__':

    
    stream = Streaming()
    
    wordcount = stream.loadData()
    
    kinesis_client = stream.createClient('kinesis', 'us-east-1')
    
#     data = stream.loadData()
    
    # send it to kinesis data stream
    stream_name = "hiring_trend_analysis_dstream"
    stream_shard_count = 1
    
    stream.sendKinesis(kinesis_client, stream_name, stream_shard_count, data) # send it!
    
#     d.to_csv('test.csv')
    
#     print(a.head())